In [2]:
import numpy as np
import h5py
import pandas as pd
import tensorflow as tf
#from tensorflow import keras
#from sklearn.model_selection import train_test_split
from glob import glob

import matplotlib.pyplot as plt 
import seaborn as sns 

plt.rcParams["figure.figsize"] = [16,9]
sns.set_style('whitegrid')

In [3]:
df_dr3_all = pd.read_csv("/data/praveen/results/dr3_full.csv")

In [6]:
unproblem_ids = pd.read_csv("/data/praveen/results/unproblem_ids.csv",index_col=0)

In [7]:
unproblem_ids.head()

,sobject_id
0,160327005601122
1,160327005601123
2,160327005601124
3,160327005601125
4,160327005601126


In [9]:
df_dr3_all.head()

,sobject_id,0,1,2,3,4,5,6,7,8,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,131116000501002,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,131116000501004,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,131116000501005,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,131116000501006,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,131116000501007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [10]:
df_temp = pd.merge(df_dr3_all,unproblem_ids, how='outer', indicator=True, on="sobject_id")

In [11]:
df_temp["_merge"].value_counts()

both          356299
left_only     232045
right_only         0
Name: _merge, dtype: int64

In [12]:
good_ids = df_temp.loc[df_temp["_merge"]=="both"]

In [13]:
good_ids.head()

,sobject_id,0,1,2,3,4,5,6,7,8,...,4450,4451,4452,4453,4454,4455,4456,4457,4458,_merge
0,131116000501002,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,both
2,131116000501005,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,both
3,131116000501006,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,both
4,131116000501007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,both
7,131116000501012,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,both


In [14]:
good_ids_data_only = good_ids.drop(["sobject_id","_merge"],1)

/tmp/ipykernel_19871/279852445.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  good_ids_data_only = good_ids.drop(["sobject_id","_merge"],1)


In [15]:
good_ids_data_only.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
df_data_only_inverted = 1 - good_ids_data_only

In [17]:
import os, sys
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.advanced_activations import PReLU
from keras import backend as K
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model

In [18]:
activation = None  # PReLU if set to None
dropout_rate = 0  # from 0 to 1
n_l_e = 5
n_epoch = 350
decoded_layer_name = 'encoded'
n_wvl = 4459
optimizer = tf.keras.optimizers.Adam()

# compute number of nodes in every connected layer
n_l_1 = int(n_wvl * 0.75)
n_l_2 = int(n_wvl * 0.50)
n_l_3 = 0  # int(n_wvl * 0.40)
n_l_4 = 0  # int(n_wvl * 0.20)
n_l_5 = int(n_wvl * 0.25)
n_l_6 = int(n_wvl * 0.10)

In [20]:
autoencoder = Sequential()

if n_l_1 > 0:
    autoencoder.add(Dense(n_l_1, input_shape=(n_wvl,), activation=activation, name='E_1'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_1'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_1'))

if n_l_2 > 0:
    autoencoder.add(Dense(n_l_2, activation=activation, name='E_2'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_2'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_2'))

if n_l_3 > 0:
    autoencoder.add(Dense(n_l_3, activation=activation, name='E_3'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_3'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_3'))

if n_l_4 > 0:
    autoencoder.add(Dense(n_l_4, activation=activation, name='E_4'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_4'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_4'))

if n_l_5 > 0:
    autoencoder.add(Dense(n_l_5, activation=activation, name='E_5'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_5'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_5'))

if n_l_6 > 0:
    autoencoder.add(Dense(n_l_6, activation=activation, name='E_6'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_6'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_6'))

autoencoder.add(Dense(n_l_e, activation=activation, name=decoded_layer_name))
if activation is None:
        autoencoder.add(PReLU(name='PR_7'))

if n_l_6 > 0:
    autoencoder.add(Dense(n_l_6, activation=activation, name='D_1'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_8'))
    if activation is None:
            autoencoder.add(PReLU(name='PR_8'))

if n_l_5 > 0:
    autoencoder.add(Dense(n_l_5, activation=activation, name='D_2'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_9'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_9'))

if n_l_4 > 0:
    autoencoder.add(Dense(n_l_4, activation=activation, name='D_3'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_10'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_10'))

if n_l_3 > 0:
    autoencoder.add(Dense(n_l_3, activation=activation, name='D_4'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_11'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_11'))

if n_l_2 > 0:
    autoencoder.add(Dense(n_l_2, activation=activation, name='D_5'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_12'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_12'))

if n_l_1 > 0:
    autoencoder.add(Dense(n_l_1, activation=activation, name='D_6'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_13'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_13'))

autoencoder.add(Dense(n_wvl, activation='linear', name='recreated'))
autoencoder.summary()

# Visualize network architecture and save the visualization as a file
#plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.pdf')
#plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.png', dpi=300)

# model file handling
out_model_file = 'model_weights.h5'

if os.path.isfile(out_model_file):
    autoencoder.load_weights(out_model_file, by_name=True)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 E_1 (Dense)                 (None, 3344)              14914240  
                                                                 
 PR_1 (PReLU)                (None, 3344)              3344      
                                                                 
 E_2 (Dense)                 (None, 2229)              7456005   
                                                                 
 PR_2 (PReLU)                (None, 2229)              2229      
                                                                 
 E_5 (Dense)                 (None, 1114)              2484220   
                                                                 
 PR_5 (PReLU)                (None, 1114)              1114      
                                                                 
 E_6 (Dense)                 (None, 445)              

In [21]:
autoencoder.compile(optimizer=optimizer, loss='mae')

In [22]:
checkpoint = ModelCheckpoint('/data/praveen/autoencoder-run-3/'+'ann_model_run_{epoch:03d}-{loss:.4f}-{val_loss:.4f}.h5',
                                     monitor='val_loss', verbose=0, save_best_only=False,
                                     save_weights_only=True, mode='auto', period=1)

In [23]:
ann_fit_hist = autoencoder.fit(df_data_only_inverted, df_data_only_inverted,
                                       epochs=n_epoch,
                                       callbacks=[checkpoint],
                                       shuffle=True,
                                       batch_size=40000,
                                       validation_split=0.10,
                                       verbose=2)

2022-05-11 11:20:01.319558: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 11438904568 exceeds 10% of free system memory.
2022-05-11 11:22:03.658147: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 11438904568 exceeds 10% of free system memory.


Epoch 1/350
9/9 - 210s - loss: 0.0343 - val_loss: 0.0267 - 210s/epoch - 23s/step
Epoch 2/350
9/9 - 169s - loss: 0.0239 - val_loss: 0.0200 - 169s/epoch - 19s/step
Epoch 3/350
9/9 - 156s - loss: 0.0187 - val_loss: 0.0187 - 156s/epoch - 17s/step
Epoch 4/350
9/9 - 168s - loss: 0.0165 - val_loss: 0.0148 - 168s/epoch - 19s/step
Epoch 5/350
9/9 - 176s - loss: 0.0144 - val_loss: 0.0136 - 176s/epoch - 20s/step
Epoch 6/350
9/9 - 179s - loss: 0.0136 - val_loss: 0.0132 - 179s/epoch - 20s/step
Epoch 7/350
9/9 - 179s - loss: 0.0132 - val_loss: 0.0129 - 179s/epoch - 20s/step
Epoch 8/350
9/9 - 179s - loss: 0.0132 - val_loss: 0.0130 - 179s/epoch - 20s/step
Epoch 9/350
9/9 - 167s - loss: 0.0128 - val_loss: 0.0128 - 167s/epoch - 19s/step
Epoch 10/350
9/9 - 180s - loss: 0.0127 - val_loss: 0.0126 - 180s/epoch - 20s/step
Epoch 11/350
9/9 - 182s - loss: 0.0126 - val_loss: 0.0126 - 182s/epoch - 20s/step
Epoch 12/350
9/9 - 176s - loss: 0.0126 - val_loss: 0.0126 - 176s/epoch - 20s/step
Epoch 13/350
9/9 - 177s -

9/9 - 171s - loss: 0.0110 - val_loss: 0.0112 - 171s/epoch - 19s/step
Epoch 102/350
9/9 - 171s - loss: 0.0111 - val_loss: 0.0110 - 171s/epoch - 19s/step
Epoch 103/350
9/9 - 178s - loss: 0.0110 - val_loss: 0.0111 - 178s/epoch - 20s/step
Epoch 104/350
9/9 - 171s - loss: 0.0110 - val_loss: 0.0111 - 171s/epoch - 19s/step
Epoch 105/350
9/9 - 167s - loss: 0.0110 - val_loss: 0.0111 - 167s/epoch - 19s/step
Epoch 106/350
9/9 - 174s - loss: 0.0111 - val_loss: 0.0111 - 174s/epoch - 19s/step
Epoch 107/350
9/9 - 178s - loss: 0.0110 - val_loss: 0.0111 - 178s/epoch - 20s/step
Epoch 108/350
9/9 - 173s - loss: 0.0110 - val_loss: 0.0111 - 173s/epoch - 19s/step
Epoch 109/350
9/9 - 174s - loss: 0.0110 - val_loss: 0.0111 - 174s/epoch - 19s/step
Epoch 110/350
9/9 - 179s - loss: 0.0110 - val_loss: 0.0111 - 179s/epoch - 20s/step
Epoch 111/350
9/9 - 172s - loss: 0.0111 - val_loss: 0.0111 - 172s/epoch - 19s/step
Epoch 112/350
9/9 - 173s - loss: 0.0110 - val_loss: 0.0111 - 173s/epoch - 19s/step
Epoch 113/350
9/9 

Epoch 200/350
9/9 - 175s - loss: 0.0108 - val_loss: 0.0107 - 175s/epoch - 19s/step
Epoch 201/350
9/9 - 176s - loss: 0.0106 - val_loss: 0.0106 - 176s/epoch - 20s/step
Epoch 202/350
9/9 - 172s - loss: 0.0106 - val_loss: 0.0106 - 172s/epoch - 19s/step
Epoch 203/350
9/9 - 175s - loss: 0.0106 - val_loss: 0.0106 - 175s/epoch - 19s/step
Epoch 204/350
9/9 - 177s - loss: 0.0106 - val_loss: 0.0106 - 177s/epoch - 20s/step
Epoch 205/350
9/9 - 168s - loss: 0.0105 - val_loss: 0.0105 - 168s/epoch - 19s/step
Epoch 206/350
9/9 - 174s - loss: 0.0106 - val_loss: 0.0107 - 174s/epoch - 19s/step
Epoch 207/350
9/9 - 176s - loss: 0.0106 - val_loss: 0.0105 - 176s/epoch - 20s/step
Epoch 208/350
9/9 - 179s - loss: 0.0105 - val_loss: 0.0106 - 179s/epoch - 20s/step
Epoch 209/350
9/9 - 182s - loss: 0.0107 - val_loss: 0.0106 - 182s/epoch - 20s/step
Epoch 210/350
9/9 - 178s - loss: 0.0106 - val_loss: 0.0106 - 178s/epoch - 20s/step
Epoch 211/350
9/9 - 178s - loss: 0.0105 - val_loss: 0.0106 - 178s/epoch - 20s/step
Epoc

Epoch 299/350
9/9 - 176s - loss: 0.0103 - val_loss: 0.0104 - 176s/epoch - 20s/step
Epoch 300/350
9/9 - 179s - loss: 0.0104 - val_loss: 0.0104 - 179s/epoch - 20s/step
Epoch 301/350
9/9 - 178s - loss: 0.0104 - val_loss: 0.0104 - 178s/epoch - 20s/step
Epoch 302/350
9/9 - 178s - loss: 0.0103 - val_loss: 0.0103 - 178s/epoch - 20s/step
Epoch 303/350
9/9 - 181s - loss: 0.0103 - val_loss: 0.0103 - 181s/epoch - 20s/step
Epoch 304/350
9/9 - 174s - loss: 0.0103 - val_loss: 0.0104 - 174s/epoch - 19s/step
Epoch 305/350
9/9 - 181s - loss: 0.0103 - val_loss: 0.0103 - 181s/epoch - 20s/step
Epoch 306/350
9/9 - 173s - loss: 0.0103 - val_loss: 0.0103 - 173s/epoch - 19s/step
Epoch 307/350
9/9 - 180s - loss: 0.0103 - val_loss: 0.0105 - 180s/epoch - 20s/step
Epoch 308/350
9/9 - 179s - loss: 0.0104 - val_loss: 0.0104 - 179s/epoch - 20s/step
Epoch 309/350
9/9 - 176s - loss: 0.0103 - val_loss: 0.0103 - 176s/epoch - 20s/step
Epoch 310/350
9/9 - 180s - loss: 0.0103 - val_loss: 0.0104 - 180s/epoch - 20s/step
Epoc

In [24]:
loss_combined = np.vstack((ann_fit_hist.history['loss'], ann_fit_hist.history['val_loss'])).T
np.savetxt('ann_network_loss.txt', loss_combined)
i_best = np.argmin(ann_fit_hist.history['val_loss'])
plt.plot(ann_fit_hist.history['loss'], label='Train')
plt.plot(ann_fit_hist.history['val_loss'], label='Validation')
plt.axvline(np.arange(n_epoch)[i_best], ls='--', color='black', alpha=0.5)
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss value')
plt.ylim(np.nanmin(loss_combined)*0.95, np.nanpercentile(loss_combined, 99))
plt.xlim(-1, n_epoch)
plt.grid(ls='--', alpha=0.2, color='black')
plt.tight_layout()
plt.legend()
plt.savefig('ann_network_loss_3.png', dpi=250)
plt.close()

In [25]:
out_model_file = 'model_weights.h5'

In [26]:
print('')
        # recover weights of the selected model and compute predictions
for i_best in [10, 25, 50, 100, 150, 200, 250, 300]:

    h5_weight_files = glob('/data/praveen/autoencoder-run-3/'+'ann_model_run_{:03.0f}-*-*.h5'.format(i_best))

    if len(h5_weight_files) == 1:
        print('----------------------------------------------------------------')
        print('Restoring epoch {:.0f} with the loss ({:.4f}).'.format(i_best, ann_fit_hist.history['val_loss'][i_best-1]))
        autoencoder.load_weights(h5_weight_files[0], by_name=True)

        sub_dir = 'epoch_{:03.0f}'.format(i_best)
        os.system('mkdir ' + sub_dir)
        os.chdir(sub_dir)

        print('Saving selected model weights')
        autoencoder.save_weights(out_model_file)

        #print('Predicting values')
        #processed_data_all = autoencoder.predict(df_data_only_inverted, verbose=2, batch_size=20)

        #plot_sample_spectra()

        #print('')
        #os.chdir('..')



----------------------------------------------------------------
Restoring epoch 10 with the loss (0.0126).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 25 with the loss (0.0126).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 50 with the loss (0.0112).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 100 with the loss (0.0110).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 150 with the loss (0.0112).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 200 with the loss (0.0107).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 250 with the loss (0.0104).
Saving selected model weights
-----------------------------

In [27]:
df_dr3 = pd.read_csv("/data/praveen/results/dr3_full.csv")

In [28]:
df_dr3.head()

,sobject_id,0,1,2,3,4,5,6,7,8,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,131116000501002,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,131116000501004,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,131116000501005,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,131116000501006,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,131116000501007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [29]:
df_dr3_data_only = df_dr3.drop(["sobject_id"],1)
df_dr3_data_only.head()
df_dr3_data_only_inverted = 1 - df_dr3_data_only

/tmp/ipykernel_19871/1450910931.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_dr3_data_only = df_dr3.drop(["sobject_id"],1)


In [30]:
df_dr3_data_only_inverted.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
print('Predicting values')
processed_data_all = autoencoder.predict(df_dr3_data_only_inverted, verbose=2, batch_size=40000)

Predicting values


2022-05-12 07:22:47.024456: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 20987407168 exceeds 10% of free system memory.


15/15 - 108s - 108s/epoch - 7s/step


2022-05-12 07:25:10.502746: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 10493703584 exceeds 10% of free system memory.


In [32]:
processed_data_all

array([[ 2.1997958e-03,  9.8226825e-05,  1.4991947e-03, ...,
        -1.7856043e-03,  2.4879040e-04, -7.3287985e-05],
       [ 3.7492020e-04,  5.6045060e-04,  2.1412917e-03, ...,
        -1.7036146e-03, -3.1532897e-04, -6.9066649e-05],
       [ 1.8160930e-03,  1.0974763e-03,  1.4585606e-03, ...,
        -1.6279925e-03, -8.0028980e-04,  5.2182085e-04],
       ...,
       [ 5.9819175e-04,  4.1774567e-04,  1.3771225e-03, ...,
        -1.4776291e-03, -8.0229889e-04,  2.2542663e-06],
       [ 3.8899994e-04,  7.4241892e-04,  1.5537017e-03, ...,
        -1.7032403e-03, -8.3542161e-04,  3.8431445e-04],
       [ 3.1395530e-04,  5.5772159e-04,  1.5193019e-03, ...,
        -1.4407267e-03, -5.8047520e-04, -3.6799931e-05]], dtype=float32)

In [33]:
df_predicted_all = pd.DataFrame(processed_data_all)

In [34]:
df_predicted_all.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,0.002200,0.000098,0.001499,0.000882,-0.000300,0.000458,0.000638,0.001344,-0.001643,-0.000947,...,-0.000540,-0.000805,-0.000936,-0.000488,-0.000110,-0.000435,0.000664,-0.001786,0.000249,-0.000073
1,0.000375,0.000560,0.002141,-0.000300,0.000356,-0.000474,-0.000065,0.000687,-0.001789,-0.000165,...,-0.000501,0.000197,-0.000166,0.000494,0.000022,-0.000314,0.000552,-0.001704,-0.000315,-0.000069
2,0.001816,0.001097,0.001459,-0.000360,0.000150,-0.000449,-0.000005,0.000237,-0.001944,-0.000709,...,-0.000544,0.000376,-0.000768,0.000507,-0.000511,-0.000685,0.000833,-0.001628,-0.000800,0.000522
3,0.001243,0.000601,0.001617,-0.000209,0.000440,-0.000274,-0.000085,0.000156,-0.001582,-0.000190,...,-0.000605,0.000211,-0.000714,0.000798,-0.000165,-0.000473,0.000779,-0.001407,-0.000620,0.000346
4,0.000385,0.000506,0.001442,-0.000253,0.000405,0.000023,-0.000049,0.000977,-0.001584,-0.000316,...,-0.000348,-0.000122,-0.000900,0.000342,-0.000520,-0.000462,0.000448,-0.001291,-0.000874,-0.000188


In [35]:
df_predicted_all.to_csv("/data/praveen/results/dr3_full_predicted_inverted_3.csv",index=None)